### Step 1: 
#### generate subduction convergence kinematics statistics 

The configuration parameters can be found in parameters.py.

Relevant parameters:
* convergence_data_filename_prefix -- the prefix of the output files
* convergence_data_filename_ext -- the extension name of the output files
* rotation_files -- location of the rotation files
* topology_files -- location of the topology files
* threshold_sampling_distance_degrees -- the default threshold sampling distance along trenches (subduction zones)
* time.start -- start time
* time.end -- end time
* time.step -- time interval between steps
* velocity_delta_time -- time interval for velocity calculation
* anchor_plate_id - the anchor plate id
* convergence_data_dir -- the name of the folder in which the output files go

The standard output can be found in out_log and the error output is in err_log.

In [1]:
import os, subprocess
from parameters import parameters

print('running python convergence.py...')
print('this may take a while, be patient...')
print('')

p = subprocess.Popen(['python', 'convergence.py'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
out, err = p.communicate()

for line in out.split('\n'):
    print(line)
    
if p.returncode !=0:
    for line in err.split('\n'):
        print(line)

print('Check the output files in ' + parameters['convergence_data_dir'])
print('file name prefix: {0}'.format(parameters['convergence_data_filename_prefix']))
print('file name extension: {0}'.format(parameters['convergence_data_filename_ext']))

running python convergence.py...
this may take a while, be patient...


Traceback (most recent call last):
  File "convergence.py", line 9, in <module>
    from subduction_convergence import subduction_convergence_over_time
ImportError: No module named subduction_convergence

Check the output files in ./convergence_data/
file name prefix: subStats
file name extension: csv


### Optional step:
#### Extract data from EarthChem data

After generating the subduction convergence kinematics statistics, the sample points of trenches can be used to extract data from EarthChem data. 
    
        Example: python extract_earth_chem.py EarthChem_all.csv output CU 5
        
This command line will extract data from locations which are within 5 degrees of a trench sample point and with a variable name CU. A shafefile(output.shp) and a csv file(output.csv) will be created for the extracted data.

For more information about the usage of extract_earth_chem.py, run "python extract_earth_chem.py -h".

The EarthChem_all.csv can be found here https://www.earthbyte.org/webdav/ftp/earthbyte/EarthChem/EarthChem_all.csv.

In [2]:
import os, subprocess

print('start extracting data....')
print('')

p = subprocess.Popen(['python', 'extract_earth_chem.py', 'EarthChem_all.csv', 'output', 'CU', '5'], 
                     stdout=subprocess.PIPE, stderr=subprocess.PIPE)
out, err = p.communicate()

for line in out.split('\n'):
    print(line)
    
if p.returncode !=0:
    for line in err.split('\n'):
        print(line)
        
if p.returncode ==0:        
    print('extract data completed successfully!')
    print('Check the following four files in the current working directory')
    print('output.csv  output.dbf  output.shp  output.shx')

start extracting data....

loading data...
./convergence_data/subStats_0.00.csv
query data...
44160
saving file...

extract data completed successfully!
Check the following four files in the current working directory
output.csv  output.dbf  output.shp  output.shx


### Step 2:
#### coregister data

This step associates locations with the trench sample points. For a given location's coordinates, this script tries to find the nearest trench sample point within a certain region. If found, the subduction convergence kinematics statistics of the trench sample point will be associated with the given location. The coregistered data will be used as input for machine learning algorithms in step 3.

Three files are created during the coregistration
* andes_real_age.csv
* andes_random_age.csv
* trench_points.csv

Data columns:

* 0 lon
* 1 lat
* 2 reconstructed lon 
* 3 reconstructed lat 
* 4 age
* 5 time
* 6 age at reconstructed location (from age grid)
* 7 subducting convergence (relative to trench) velocity magnitude (in cm/yr)
* 8 subducting convergence velocity obliquity angle (angle between trench normal vector and convergence velocity vector)
* 9 trench absolute (relative to anchor plate) velocity magnitude (in cm/yr)
* 10 trench absolute velocity obliquity angle (angle between trench normal vector and trench absolute velocity vector)
* 11 length of arc segment (in degrees) that current point is on
* 12 trench normal azimuth angle (clockwise starting at North, ie, 0 to 360 degrees) at current point
* 13 subducting plate ID
* 14 trench plate ID
* 15 distance (in degrees) along the trench line to the nearest trench edge
* 16 the distance (in degrees) along the trench line from the start edge of the trench
* 17 convergence velocity orthogonal (in cm/yr)
* 18 convergence velocity parallel  (in cm/yr) 
* 19 the trench plate absolute velocity orthogonal (in cm/yr)
* 20 the trench plate absolute velocity orthogonal (in cm/yr)
* 21 the subducting plate absolute velocity magnitude (in cm/yr)
* 22 the subducting plate absolute velocityobliquity angle (in degrees)
* 23 the subducting plate absolute velocity orthogonal       
* 24 the subducting plate absolute velocity parallel
* 25 plate id of the input point

In [3]:
import os, subprocess

print('start coregistration....')
print('this may take a while, be patient...')
print('')

p = subprocess.Popen(['python', 'coregistration.py'], 
                     stdout=subprocess.PIPE, stderr=subprocess.PIPE)
out, err = p.communicate()

for line in out.split('\n'):
    print(line)
    
if p.returncode !=0:
    for line in err.split('\n'):
        print(line)
        
if p.returncode ==0:        
    print('coregistration completed successfully!')
    print('check the following three files in the current working directory')
    print('andes_random_age.csv  andes_real_age.csv trench_points.csv')

start coregistration....
this may take a while, be patient...

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
('Time taken:', 80.25169610977173, ' seconds')

coregistration completed successfully!
check the following three files in t

### Step 3:

#### Machine learning

Open machine_learning.ipynb and following the instructions inside 